Imports and Setup

In [ ]:
!pip -q install wandb

import os, random, time
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import wandb

from pathlib import Path
from torch.utils.data import DataLoader

from src.datasets import AssessmentPairs, train_val_split, AssessmentTorchDataset, stratified_subsample, class_counts


In [ ]:
def seed_all(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

seed_all(42)
device = "cuda" if torch.cuda.is_available() else "cpu"
device


Load data

In [ ]:
ROOT = Path("/content/drive/MyDrive/Hands-on-CV-Project2/data/assessment")

pairs = AssessmentPairs(ROOT).load_pairs()
pairs_sub = stratified_subsample(pairs, frac=0.10, seed=42)  # start with 10%, increase later

train_pairs, val_pairs = train_val_split(pairs_sub, val_ratio=0.2, seed=42)

train_ds = AssessmentTorchDataset(train_pairs)
val_ds   = AssessmentTorchDataset(val_pairs)

train_loader = DataLoader(train_ds, batch_size=128, shuffle=True, num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_ds, batch_size=128, shuffle=False, num_workers=2, pin_memory=True)

batch = next(iter(train_loader))
rgb_in_ch = batch["rgb"].shape[1]
lidar_in_ch = batch["lidar"].shape[1]
print("rgb_in_ch:", rgb_in_ch, "lidar_in_ch:", lidar_in_ch)


Train model

In [ ]:
wandb.login()
run = wandb.init(project="cilp-extended-assessment", name="task5_cilp_contrastive", config={
    "task": 5,
    "lr": 1e-3,
    "epochs": 20,
    "batch_size": train_loader.batch_size,
    "subset_frac": 0.10,
    "temperature": 0.07,
})

# Choose embedder type based on Task 4 result:
# - For contrastive: normalize=True
rgb_embed = Embedder(rgb_in_ch, emb_size=200, normalize=True)         # or EmbedderStrided(...)
lid_embed = Embedder(lidar_in_ch, emb_size=200, normalize=True)       # or EmbedderStrided(...)

cilp = CILP(rgb_embed, lid_embed, temperature=0.07)
cilp = train_cilp(cilp, train_loader, val_loader, device=device, epochs=20, lr=1e-3, wandb_run=run)
run.finish()


Train Projetor

In [ ]:
run = wandb.init(project="cilp-extended-assessment", name="task5_projector", config={
    "task": 5,
    "epochs": 20,
    "lr": 1e-3,
    "dim": 200,
})

proj = Projector(dim=200)
proj = train_projector(cilp, proj, train_loader, val_loader, device=device, epochs=20, lr=1e-3, wandb_run=run)
run.finish()


Train final Classifier

In [ ]:
run = wandb.init(project="cilp-extended-assessment", name="task5_rgb_to_lidar_classifier", config={
    "task": 5,
    "epochs": 10,
    "lr": 1e-3,
})

clf = EmbeddingClassifier(dim=200)
clf = train_embedding_classifier(cilp, proj, clf, train_loader, val_loader, device=device, epochs=10, lr=1e-3, wandb_run=run)
run.finish()


Testing

In [ ]:
@torch.no_grad()
def test_5_batches(cilp, proj, clf, loader, device):
    cilp.eval(); proj.eval(); clf.eval()
    correct, total = 0, 0
    it = iter(loader)
    for i in range(5):
        batch = next(it)
        rgb = batch["rgb"].to(device)
        y = batch["y"].to(device)
        z = cilp.rgb(rgb)
        z_hat = proj(z)
        pred = torch.argmax(clf(z_hat), dim=1)
        correct += (pred == y).sum().item()
        total += y.numel()
    acc = correct / total
    print("Accuracy over 5 val batches:", acc)
    return acc

test_5_batches(cilp, proj, clf, val_loader, device)


Save the checkpoint

In [ ]:
from pathlib import Path
Path("checkpoints").mkdir(exist_ok=True)

torch.save(cilp.state_dict(), "checkpoints/cilp_contrastive.pt")
torch.save(proj.state_dict(), "checkpoints/projector_rgb_to_lidar.pt")
torch.save(clf.state_dict(), "checkpoints/classifier_rgb_to_lidar.pt")
